In [2]:
import numpy as np
import torch
import torch.nn.functional as F
import os




####################################
def retrieve_elements_from_indices(tensor, indices):
    flattened_tensor = tensor.flatten(start_dim=2)
    output = flattened_tensor.gather(dim=2, index=indices.flatten(start_dim=2)).view_as(indices)
    return output

##############
count = 0
# -------------------------------------------------------------------------
# Handy Utilities
# -------------------------------------------------------------------------
def to_polar_np(velo):
    if len(velo.shape) == 4:
        velo = velo.transpose(1, 2, 3, 0)

    if velo.shape[2] > 4:
        assert velo.shape[0] <= 4
        velo = velo.transpose(1, 2, 0, 3)
        switch=True
    else:
        switch=False
    print("inside to velo")
#     print(velo[:,:,3][(velo[:,:,3]!=0)&(velo[:,:,3]!=1)])
    
    # assumes r x n/r x (3,4) velo
    dist = np.sqrt(velo[:, :, 0] ** 2 + velo[:, :, 1] ** 2)
    # theta = np.arctan2(velo[:, 1], velo[:, 0])
    out = np.stack([dist, velo[:, :, 2],velo[:,:,3]], axis=2)
    #out = np.stack([dist, velo[:, :, 2]], axis=2)
    
    if switch:
        out = out.transpose(2, 0, 1, 3)

    if len(velo.shape) == 4: 
        out = out.transpose(3, 0, 1, 2)
    
    
#     print(dist.shape)
#     print("out ki shape")
#     print(out.shape)
    
    return out

def to_polar(velo):
    if len(velo.shape) == 4:
        velo = velo.permute(1, 2, 3, 0)

    if velo.shape[2] > 4:
        assert velo.shape[0] <= 4
        velo = velo.permute(1, 2, 0, 3)
        switch=True
    else:
        switch=False
    
    # assumes r x n/r x (3,4) velo
    dist = torch.sqrt(velo[:, :, 0] ** 2 + velo[:, :, 1] ** 2)
    # theta = np.arctan2(velo[:, 1], velo[:, 0])
    
   
    
    out = torch.stack([dist, velo[:, :, 2]], dim=2)
    
    if switch:
        out = out.permute(2, 0, 1, 3)

    if len(velo.shape) == 4: 
        out = out.permute(3, 0, 1, 2)
    
    return out

def from_polar(velo):
    angles = np.linspace(0, np.pi * 2, velo.shape[-1])
    dist, z = velo[:, 0], velo[:, 1]
    x = torch.Tensor(np.cos(angles)).cuda().unsqueeze(0).unsqueeze(0) * dist
    y = torch.Tensor(np.sin(angles)).cuda().unsqueeze(0).unsqueeze(0) * dist
    out = torch.stack([x,y,z], dim=1)

    return out

def from_polar_np(velo):
    angles = np.linspace(0, np.pi * 2, velo.shape[-1])
    dist, z = velo[:, 0], velo[:, 1]
    x = np.cos(angles) * dist
    y = np.sin(angles) * dist
    out = np.stack([x,y,z], axis=1)
    return out.astype('float32')

def print_and_log_scalar(writer, name, value, write_no, end_token=''):
    if isinstance(value, list):
        if len(value) == 0: return 
        value = torch.mean(torch.stack(value))
    zeros = 40 - len(name) 
    name += ' ' * zeros
    print('{} @ write {} = {:.4f}{}'.format(name, write_no, value, end_token))
    writer.add_scalar(name, value, write_no)

def log_point_clouds(writer, data, name, step):
    if len(data.shape) == 3:
        data = [data]
    
    out = np.stack([from_polar(x.transpose(1, 2, 0)) for x in \
            data.cpu().data.numpy()])
    out = torch.tensor(out).float()

    for i, cloud in enumerate(out):
        cloud = cloud.view(-1, 3)
        writer.add_embedding(cloud, tag=name + '_%d' % i, global_step=step)

def print_and_save_args(args, path):
    print(args)
    # let's save the args as json to enable easy loading
    import json
    with open(os.path.join(path, 'args.json'), 'w') as f: 
        json.dump(vars(args), f)

def maybe_create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

def remove_zeros(pc):
  
    
    xx = torch.cuda.FloatTensor(pc)
#     print(xx.dim())
    if xx.dim() == 3: 
        xx = xx.unsqueeze(0)
    
    print("xx ki shape")
    print(xx.shape)
    
    
#     print("xx ki poll khulegi")
#     print(xx[:,2][(xx[:,2]!=0)&(xx[:,2]!=1)])
#     print(xx[:,2].unsqueeze(0).shape)
   
    iters = 0
    pad = 2
    ks = 5
    while (xx[:, 0] == 0).sum() > 0 : 
        print(iters)
        print ("The number of zeros are:", (xx[:, 0] == 0).sum())
        if iters  > 100:
            raise ValueError()
            ks += 2
            pad += 1
#             print("iters if ke andar gya")
#         print("maa_ka_chodi")
        
        mask = (xx[:, 0] == 0).unsqueeze(1).float()
        
#         print("mask is")
#         print(mask)
        
#         print("mask ka shape")
#         print(mask.shape)
        
        out_a,indices = F.max_pool2d(xx[:, 0], ks, padding=pad, stride=1,return_indices=True)
#         print(indices)
#         print(indices.shape)
#         out_b = -F.max_pool2d(-xx[:, 1], ks, padding=pad, stride=1)
        out_b = F.max_pool2d(xx[:, 1], ks, padding=pad, stride=1)
        out_c = retrieve_elements_from_indices(xx[:,2].unsqueeze(0), indices)
        
        print("out_b shapee before")
        print(out_b.shape)
        
        # Changing this since our z = -z from carla
        #out_b_ = (xx[:, 1]).min(dim=-1, keepdim=True)[0].expand_as(out_b)
        #out_b = torch.cat([out_b_[:, :10], out_b[:, 10:]], dim=1)
        out_b = out_b.expand_as(out_a)
        
        print("out_b shapee after")
        print(out_b.shape)
        
#         print(out_c.shape)
      
        
        out = torch.stack([out_a, out_b,out_c.squeeze(0)], dim=1)
#         out = torch.stack([out_a, out_b], dim=1)
        
        mask = (xx[:, 0] == 0).unsqueeze(1)
        mask = mask.float()
        
        print("mask ki shape")
        print(mask.shape)
        print("out ki shape")
        print(out.shape)
        print("out_c mein number of ones")
        print(out_c[out_c==1].shape)
        
        print("pehla:",(xx[:,2][xx[:,2]==1].size()))
        xx = xx * (1 - mask) + (mask) * out
        
        
        print("doosra:",(xx[:,2][xx[:,2]==1].size()))
        iters += 1
    
#     print("bahar aaj ja aa bhi ab")
    return xx.cpu().data.numpy()

# Hello

# I read this paper recently and wanted to try my hands on it. Thank you for providing the code for the same.
# I have one doubt with regards to the preprocessing step.

# The preprocessing step should return a tensor of shape (N, 60, 512, 3) but on trying the code, the tensor comes out to be of shape (N, 40, 256, 3)
# Can you clarify whether it's okay to have this output tensor? Also, can you shed some light on the following two lines mentioned in the preprocessing function defined in utils.py

# Line 151: dataset = dataset[:, 5:45]
# Line 180: return dataset[:, :, :, ::2]
# I think that line 151 is reducing the circles or clusters from 60 to 40 and then line 180 is reducing the point per circle/cluster to 256 from 512.

# If it's okay to have this as the output tensor, is there any reasoning for reducing clusters and points per cluster?

# Please help.

# Thanks
# Abhishek

# @pclucas14
 
# Owner
# pclucas14 commented 18 days ago
# Hi Abhishek,

# What you said is correct. I downsampled the input mainly to reduce the computational burden and improve training speed. I also discarded the outer circles because they usually are the ones that contain the most noise, and thus are the hardest to model. You can bypass these steps if you want, however you may need to make a few changes in the model architecture.

# Best,
# Lucas



def preprocess(dataset, lidar_range):
    
#     dataset=np.array(dataset,dtype=np.float32)
    
    dataset[:,:,:,:3]=dataset[:,:,:,:3]/lidar_range   # Max LIDAR value
    
    dataset = to_polar_np(dataset).transpose(0, 3, 1, 2)
    print("dataset ki shape")
    print(dataset.shape)
    previous = (dataset[:, 0] == 0).sum()

    remove = []
    for i in range(dataset.shape[0]):
#         try:
        print("the trun going on is :",i)
        pp = remove_zeros(dataset[i]).squeeze(0)
#         print("pp ki maa ki aankh")
#             print(pp.shape)
        dataset[i] = pp
#         except:
#             remove += [i]
#             print("exception aa gya ohoho")

    for i in remove:
        dataset = np.concatenate([dataset[:i-1], dataset[i+1:]], axis=0)
        print("yeh_kyun ho rha hain")
    return dataset

# def preprocess(dataset, lidar_range):
#     # remove outliers 
#     #min_a, max_a = np.percentile(dataset[:, :, :, [0]], 1), np.percentile(dataset[:, :, :, [0]], 99)
#     #min_b, max_b = np.percentile(dataset[:, :, :, [1]], 1), np.percentile(dataset[:, :, :, [1]], 99)
#     #min_c, max_c = np.percentile(dataset[:, :, :, [2]], 1), np.percentile(dataset[:, :, :, [2]], 99)
#     # min_a, max_a = -41.1245002746582,   36.833248138427734
#     # min_b, max_b = -25.833599090576172, 30.474000930786133
#     # min_c, max_c = -2.3989999294281006, 0.7383332848548889
#     #changes
#     min_a, max_a = -71.1245002746582,   66.833248138427734
#     min_b, max_b = -55.833599090576172, 60.474000930786133
#     min_c, max_c = -30.3989999294281006, 10.7383332848548889


#     #changed, commented this, now at phi instead of 40 getting all 60
#     #dataset = dataset[:, 5:45]


#     mask = np.maximum(dataset[:, :, :, 0] < min_a, dataset[:, :, :, 0] > max_a)
#     mask = np.maximum(mask, np.maximum(dataset[:, :, :, 1] < min_b, dataset[:, :, :, 1] > max_b))
#     mask = np.maximum(mask, np.maximum(dataset[:, :, :, 2] < min_c, dataset[:, :, :, 2] > max_c))
    
#     dist = dataset[:, :, :, 0] ** 2 + dataset[:, :, :, 1] ** 2
#     mask = np.maximum(mask, dist < 7)

#     dataset = dataset * (1 - np.expand_dims(mask, -1))

#     #----------------------------------------------------------------------

#     #Fix for an erro where np.absolute(dataset).max()   goes to 0 causing failure
#     # npmaxError=0.0
#     # if np.absolute(dataset).max()==0.0:
#     #     npmaxError=1.0

#     #----------------------------------------------------------------------

#     # dataset /= (np.absolute(dataset).max()+npmaxError)
#     dataset /= lidar_range   # Max LIDAR value
#     # normalization_factor = (np.absolute(dataset).max()+npmaxError)

#     dataset = to_polar_np(dataset).transpose(0, 3, 1, 2)
#     previous = (dataset[:, 0] == 0).sum()

#     remove = []
#     for i in range(dataset.shape[0]):
#         #print('processing {}/{}'.format(i, dataset.shape[0]))
#         try:
#             pp = remove_zeros(dataset[i]).squeeze(0)
#             dataset[i] = pp
#         except:
#             #print('removing %d' % i)
#             remove += [i]

#     for i in remove:
#         dataset = np.concatenate([dataset[:i-1], dataset[i+1:]], axis=0)
#     #print(dataset[:, :, :, ::2].shape)


#     #changed,  for every 2 entries it retuend 1 entry thereby reducing the 512 to 256
#     # return dataset[:, :, :, ::2]
#     # print('In preprocess')
#     # print(dataset.shape)
#     # if give_factor:
#     #     return dataset, normalization_factor
#     # else:
#     return dataset


def show_pc(velo, save=0, save_path=None):
    import mayavi.mlab

    fig = mayavi.mlab.figure(size=(1400, 700), bgcolor=(0,0,0)) 

    if len(velo.shape) == 3:
        if velo.shape[0] == 3 : 
            velo = velo.transpose(1,2,0)

        assert velo.shape[2] == 3
        velo = velo.reshape((-1, 3))

    max_ = np.absolute(velo[:, :2]).max()
    nodes = mayavi.mlab.points3d(
        velo[:, 0],   # x
        velo[:, 1],   # y
        velo[:, 2],   # z
        scale_factor=0.008, #0.022,     # scale of the points
        figure=fig) 
    
    nodes.glyph.scale_mode = 'scale_by_vector'
    color = (velo[:, 2] - velo[:, 2].min()) / (velo[:, 2].max() - velo[:, 2].min())
    color = (velo[:, 2] - -0.069667026) / ( 0.0041348818 - -0.069667026)
    
    nodes.mlab_source.dataset.point_data.scalars = color
    print('showing pc')
    aa, bb = -95, -40 #np.random.randint(-105, -85), np.random.randint(-55, -35)
    print(aa, bb)
    mayavi.mlab.view(azimuth=-87, elevation=-40, focalpoint=(0, 0, np.median(velo[:, -1])))
    f = mayavi.mlab.gcf()
    f.scene.camera.zoom(2.7)

    if save:
        print(save)
        mayavi.mlab.savefig('../inter_images_2/{}.png'.format(i))
        mayavi.mlab.close()
    elif save_path is not None:
        mayavi.mlab.savefig(save_path)
        mayavi.mlab.close()
    else:
        mayavi.mlab.show()

def show_pc_lite(velo, ind=1, show=True):
    velo=velo.cpu()
    # print(velo.shape)
    import matplotlib.pyplot as plt
    plt.scatter(velo[:, 0], velo[:, 1], s=0.7, color='k')
    plt.show() 


def to_attr(args_dict):
    class AttrDict(dict):
        def __init__(self, *args, **kwargs):
            super(AttrDict, self).__init__(*args, **kwargs)
            self.__dict__ = self

    return AttrDict(args_dict)


def load_model_from_file(path, epoch, model='dis'):
    from models import netD, netG, VAE
    import json
    with open(os.path.join(path, 'args.json'), 'r') as f: 
        old_args = json.load(f)

    old_args = to_attr(old_args)
    if 'gen' in model.lower():
        try:
            z_ = old_args.z_dim
            model_ = VAE(old_args)
        except:
            z_ = 100
            model_ = netG(old_args, nz=z_, nc= 3 if old_args.no_polar else 2)
    elif 'dis' in model.lower():
        model_ = netD(old_args)
    else: 
        raise ValueError('%s is not a valid model name' % model)

    model_.load_state_dict(torch.load(os.path.join(path, 'models/%s_%d.pth' % (model, epoch))))
    print('model successfully loaded')

    return model_, epoch 


def batch_pairwise_dist(A, B):
    # pa, pb are bs x points x 3
    r_A = (A * A).sum(dim=2, keepdim=True)
    r_B = (B * B).sum(dim=2, keepdim=True)
    m = torch.bmm(A, B.permute(0, 2, 1))
    D = r_A - 2 * m + r_B.permute(0, 2, 1)
    return D

def chamfer_quadratic(a,b):
    D = batch_pairwise_dist(a,b)
    return D.min(dim=-1)[0], D.min(dim=-2)[0]


# Utilities for baseline
def get_chamfer_dist(get_slow=False):
    try:
        if get_slow: raise ValueError

        import sys
        sys.path.insert(0, './nndistance')
        from modules.nnd import NNDModule
        dist = NNDModule()
    except:
        dist = chamfer_quadratic

    def loss(a, b):
        if a.dim() == 4:
            if a.size(1) == 2: 
                a = from_polar(a)

            assert a.size(1) == 3
            a = a.permute(0, 2, 3, 1).contiguous().reshape(a.size(0), -1, 3)
            
        if b.dim() == 4:
            if b.size(1) == 2: 
                b = from_polar(b)

            assert b.size(1) == 3
            b = b.permute(0, 2, 3, 1).contiguous().reshape(b.size(0), -1, 3)

        assert a.dim() == b.dim() == 3
        if a.size(-1) != 3: 
            assert a.size(-2) == 3
            a = a.transpose(-2, -1).contiguous()
        
        if b.size(-1) != 3: 
            assert b.size(-2) == 3
            b = a.transpose(-2, -1).contiguous()

        dist_a, dist_b = dist(a, b)
        return dist_a.sum(dim=-1) + dist_b.sum(dim=-1)

    return loss


# if __name__ == '__main__':
#     import pdb; pdb.set_trace()
#     # check if both chamfer implementations give the same results
#     ch_fast = get_chamfer_dist()
#     ch_slow = get_chamfer_dist(get_slow=True)

#     for _ in range(10):
#         x = torch.cuda.FloatTensor(32, 1000, 3).normal_()
#         y = torch.cuda.FloatTensor(32, 1000, 3).normal_()
        
#         out_fast = ch_fast(x,y)
#         out_slow = ch_slow(x,y)



In [3]:
STATIC_TRAIN_FOLDER_PATH  = '/home/saby/Projects/ati/data/data/datasets/Carla/16beam-Data/small_map/validate_idea/dynamic_24/_segment_out_npy/'
static_dataset_train = np.load(os.path.join(STATIC_TRAIN_FOLDER_PATH,"0.npy"))
print(static_dataset_train.shape)
static_dataset_train[:,:,:,:][(static_dataset_train[:,:,:,3]!=0)&(static_dataset_train[:,:,:,3]!=1)]=0
# static_dataset_train = preprocess(static_dataset_train,100).astype('float32')

(3056, 16, 1024, 4)


In [ ]:
validation = np.load(os.path.join(STATIC_TRAIN_FOLDER_PATH,"0.npy"))
# print(static_dataset_train[12,2,:,:][(static_dataset_train[12,2,:,:]!=1)&(static_dataset_train[12,2,:,:]!=0)])

print(static_dataset_train[2,0,:,:][static_dataset_train[2,2,:,:]==1].shape)
print(static_dataset_train[2,2,:,:][static_dataset_train[2,2,:,:]==1].shape)


# final=np.array(static_dataset_train[10,2:,:])
# print(final.sum())

validation[:,:,:,:][(validation[:,:,:,3]!=0)&(validation[:,:,:,3]!=1)]=0
print(validation[2,:,:,:][validation[2,:,:,3]==1].shape)

# final_2=np.array(validation[10,:,:,3])
# print(final_2.sum())

# print(static_dataset_train[10,2,:,:][(static_dataset_train[10,2,:,:]!=1)&(static_dataset_train[10,2,:,:]!=0)])
# print(validation[10,:,:,3])


# print("pehla:",set(zip(results[0],results[1])))
# validation=np.where(validation[0,:,:,3]==1)
# print("doosra:",set(zip(validation[0],validation[1])))



array_1=validation
array_2=static_dataset_train


In [ ]:
array_1.shape

In [ ]:
array_2.shape

In [ ]:
idx = np.random.choice(array_1.shape[0],1)[0]
idx

In [ ]:
before_img = array_1[idx]
after_img  = array_2[idx]

In [ ]:
after_img_3d = from_polar_np(np.array([after_img[:2,:,:]]))[0]

In [ ]:
after_img_4d = np.concatenate((after_img_3d, np.array([after_img[2]])), axis=0).transpose(1,2,0)

In [ ]:
before_img.shape, after_img.shape, after_img_4d.shape

In [ ]:
before_pts_arr = before_img.reshape(-1,4)[:,:3]
before_color_arr = np.array([before_img.reshape(-1,4)[:,3], np.zeros(16*1024), np.zeros(16*1024)]).T


after_pts_arr = after_img_4d.reshape(-1,4)[:,:3]
after_color_arr = np.array([after_img_4d.reshape(-1,4)[:,3], np.zeros(16*1024), np.zeros(16*1024)]).T

In [ ]:
import open3d as o3d
# o3d.visualization.draw_geometries([pcd_patanahi,pcd_dynamic,pcd_static],width=1280, height=800)

In [ ]:
before_pcd = o3d.geometry.PointCloud()
before_pcd.points = o3d.utility.Vector3dVector(before_pts_arr)
before_pcd.colors = o3d.utility.Vector3dVector(before_color_arr)

after_pcd = o3d.geometry.PointCloud()
after_pcd.points = o3d.utility.Vector3dVector(after_pts_arr)
after_pcd.colors = o3d.utility.Vector3dVector(after_color_arr)

In [ ]:
def draw_pcd(pcd, where='opn_nb'):
    if where is 'opn_nb':
        visualizer = o3d.JVisualizer()
        visualizer.add_geometry(pcd)
        visualizer.show()
    elif where is 'opn_view':
        o3d.visualization.draw_geometries([pcd], width=1280, height=800)
    elif where is 'mat_3d':
        plt.figure()
        pts = np.asarray(pcd.points)
        plt.scatter(pts[:,0], pts[:,1], pts[:,2])
        plt.grid()
        plt.show()
    elif where is 'mat_2d':
        plt.figure()
        pts = np.asarray(pcd.points)
        plt.scatter(pts[:,0], pts[:,1])
        plt.grid()
        plt.show()

In [ ]:
draw_pcd(before_pcd)

In [ ]:
draw_pcd(after_pcd)

In [ ]:
a=torch.randn(1,1,10,10)
b=torch.randn(1,1,10,10)

In [ ]:
print(a)
print(b)
pad=1
ks=2
out_a,indices = F.max_pool2d(a, ks, padding=pad, stride=1,return_indices=True)
print(out_a)
print(indices)
output=retrieve_elements_from_indices(b,indices)
print(output)

In [6]:
import torch
import torch.nn as nn

In [42]:
loss = nn.CrossEntropyLoss()
input_1 = torch.randn(1,2, 16,1024, requires_grad=True)
target = torch.zeros(1,2,16 ,1024,dtype=torch.long)
target_2 = torch.zeros(1,2,16 ,1024)
print(input_1.squeeze(0).shape)
print(target)

output = loss(input_1, target[:,0])
# output.backward()

print (output)

torch.Size([2, 16, 1024])
tensor([[[[0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          ...,
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0]],

         [[0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          ...,
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0]]]])
tensor(0.9009, grad_fn=<NllLoss2DBackward>)
